In [120]:
import sys
import os

import numpy as np
import pandas as pd
import geopandas as gpd
import fiona

import rioxarray  # Surface data manipulation
import xarray  # Surface data manipulation

from pysal.explore import esda  # Exploratory Spatial analytics
from pysal.lib import weights  # Spatial weights

import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt 
from matplotlib import colors
from matplotlib.lines import Line2D

import seaborn 
import contextily  # Background tiles

In [65]:
target_cd = 'c:\\Users\\jarem\\OneDrive - London School of Economics\\YEAR 2\\1. Policy paper\\policy-paper-repo'
os.chdir(target_cd)
print(f"Current working directory path set to: {os.getcwd()}")

Current working directory path set to: c:\Users\jarem\OneDrive - London School of Economics\YEAR 2\1. Policy paper\policy-paper-repo


In [66]:
df = gpd.read_file('data/clean/outcome/NUTS_3/GDP_per_capita_NUTS3_2002_23.gpkg', layer='nuts3')

In [67]:
df_filtered = df.copy()
df_filtered = df_filtered[df_filtered['INDICATOR'].isin(
    ['produkt krajowy brutto na 1 mieszkańca',
    'produkt krajowy brutto na 1 mieszkańca, Polska = 100',
    'produkt krajowy brutto na 1 mieszkańca, region=100',])]

df_filtered.drop(columns=['NAME_LATN', 'MOUNT_TYPE', 'COAST_TYPE'], inplace=True)
df_filtered = df_filtered[df_filtered['YEAR']<2024]

In [68]:
df_filtered.INDICATOR.unique()

array(['produkt krajowy brutto na 1 mieszkańca',
       'produkt krajowy brutto na 1 mieszkańca, Polska = 100',
       'produkt krajowy brutto na 1 mieszkańca, region=100'], dtype=object)

In [69]:
# rename col names for clarity
df_filtered['VAR_DESCR'] = df_filtered['INDICATOR']
rename_dict = {
    'produkt krajowy brutto na 1 mieszkańca': 'GDP_PC',
    'produkt krajowy brutto na 1 mieszkańca, Polska = 100': 'GDP_PC_PL_100',
    'produkt krajowy brutto na 1 mieszkańca, region=100': 'GDP_PC_REG_100'
}
df_filtered['INDICATOR'] = df_filtered['INDICATOR'].map(rename_dict)


In [ ]:
# log-transformed GDP per capita
log_gdp = df_filtered[df_filtered['INDICATOR'] == 'GDP_PC'].copy()
log_gdp['VALUE'] = np.log(log_gdp['VALUE'])
log_gdp['INDICATOR'] = 'LOG_GDP_PC'
log_gdp['VAR_DESCR'] = 'Log transformed gdp per capita'

# Append to the original DataFrame
df_filtered = pd.concat([df_filtered, log_gdp], ignore_index=True)

In [84]:
gdf_filtered = gpd.GeoDataFrame(df_filtered, geometry='geometry')
gdf_filtered.to_file('data/clean/outcome/NUTS_3/GDP_per_capita_NUTS3_2002_23_log_transformed.gpkg', layer='nuts3', driver='GPKG')

In [71]:
# Pivot to wide format with flat column names
df_wide = df_filtered.pivot(index='NUTS_NAME', columns=['INDICATOR', 'YEAR'], values='VALUE')
df_wide.columns = [f"{indicator}_{year}" for indicator, year in df_wide.columns]
df_wide = df_wide.reset_index()

In [83]:
# df_wide.index, df_wide.columns

In [74]:
df_target = df_filtered.loc[:, ['NUTS_NAME','CODE_PL', 'NUTS_ID', 'URBN_TYPE','geometry']]
df_target = df_target.drop_duplicates(subset='NUTS_NAME')

In [75]:
# Merge wide columns back to df_filtered (on NUTS_NAME)
df_merged = pd.merge(df_wide, df_target,on='NUTS_NAME')

In [77]:
gdf_merged = gpd.GeoDataFrame(df_merged, geometry='geometry')
gdf_merged.to_file('data/clean/outcome/NUTS_3/GDP_per_capita_NUTS3_2002_23_wide.gpkg', layer='nuts3', driver='GPKG')

### LISA 

In [141]:
import geopandas as gpd
import pandas as pd
from libpysal.weights import Queen
from esda.moran import Moran_Local
from esda.moran import Moran
from libpysal.weights import lag_spatial


In [121]:
gdf = gpd.read_file("data/clean/outcome/NUTS_3/GDP_per_capita_NUTS3_2002_23_log_transformed.gpkg", layer='nuts3')

In [122]:
gdf = gdf.sort_values(["YEAR", "NUTS_ID"]).reset_index(drop=True)


### Create spatial weights

In [123]:
gdf = gdf.set_geometry("geometry")

gdf_wide = (
    gdf
    .pivot_table(
        index=["NUTS_ID", "YEAR", "geometry"],
        columns="INDICATOR",
        values="VALUE"
    )
    .reset_index()
)

gdf_wide = gpd.GeoDataFrame(
    gdf_wide,
    geometry="geometry",
    crs=gdf.crs
)

In [ ]:
# COMPUTE WEIGHTS
gdf_regions = gdf_wide.drop_duplicates("NUTS_ID")

from libpysal.weights import KNN, Rook

# fallback for isolated polygons
w = KNN.from_dataframe(gdf, k=4)

# w = Queen.from_dataframe(gdf_regions, ids="NUTS_ID", geom_col='geometry')
w.transform = "R"


### Compute LISA

In [ ]:
# output location
BASE_OUT = r"C:\Users\jarem\OneDrive - London School of Economics\YEAR 2\1. Policy paper\policy-paper-repo\outputs\LISA"

In [ ]:
### Helpers for LISA

CLUSTER_MAP = {
    1: "HH",
    2: "LH",
    3: "LL",
    4: "HL"
}

In [127]:
def run_lisa(df, variable, w):
    x = df[variable].values

    lisa = Moran_Local(x, w, permutations=999)

    out = df.copy()
    out["variable"] = variable
    out["local_I"] = lisa.Is
    out["p_value"] = lisa.p_sim
    out["z_score"] = lisa.z_sim
    out["cluster_code"] = lisa.q
    out["cluster"] = out["cluster_code"].map(CLUSTER_MAP)
    out["significant"] = out["p_value"] < 0.05

    # mark non-significant
    out.loc[~out["significant"], "cluster"] = "NS"

    return out


In [128]:
### MAP RESULTS
def plot_lisa_map(gdf, variable, year, out_path):
    colors = {
        "HH": "#d7191c",   # red
        "LL": "#2c7bb6",   # blue
        "HL": "#fdae61",   # orange
        "LH": "#abd9e9",   # light blue
        "NS": "#eeeeee"    # grey
    }

    labels = {
        "HH": "High–High",
        "LL": "Low–Low",
        "HL": "High–Low",
        "LH": "Low–High",
        "NS": "Not significant"
    }

    fig, ax = plt.subplots(1, 1, figsize=(8, 8))

    gdf.plot(
        color=gdf["cluster"].map(colors),
        linewidth=0.3,
        edgecolor="black",
        ax=ax
    )

    # ---- CUSTOM LEGEND ----
    legend_elements = [
        Line2D(
            [0], [0],
            marker='s',
            color='w',
            label=labels[k],
            markerfacecolor=colors[k],
            markersize=12
        )
        for k in ["HH", "LL", "HL", "LH", "NS"]
    ]

    ax.legend(
        handles=legend_elements,
        title="LISA clusters",
        loc="lower left",
        frameon=True
    )

    ax.set_title(f"LISA – {variable} ({year})", fontsize=12)
    ax.axis("off")

    plt.tight_layout()
    plt.savefig(out_path, dpi=300)
    plt.close()


In [151]:
### QUADRANT PLOT

import matplotlib.pyplot as plt
from libpysal.weights import lag_spatial
from esda import Moran, Moran_Local
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_lisa_scatter(df, variable, w, year, out_path):
    # ---- Standardize variable ----
    x = df[variable].values
    x_std = (x - x.mean()) / x.std()

    # ---- Spatial lag ----
    w_lag = lag_spatial(w, x_std)

    # ---- Global Moran's I ----
    moran_global = Moran(x_std, w, permutations=999)
    I_global = moran_global.I
    p_global = moran_global.p_sim

    fig, ax = plt.subplots(figsize=(6, 6))

    # ---- Scatter points ----
    ax.scatter(
        x_std,
        w_lag,
        s=25,
        color="black",
        alpha=0.6
    )

    # ---- Reference lines ----
    ax.axhline(0, color="black", linewidth=1)
    ax.axvline(0, color="black", linewidth=1)

    # ---- Moran regression line (slope = Global I) ----
    x_vals = np.array([x_std.min(), x_std.max()])
    ax.plot(
        x_vals,
        I_global * x_vals,
        color="black",
        linewidth=1
    )

    # ---- Quadrant labels ----
    ax.text(0.65, 0.85, "High–High\n(Li +)", transform=ax.transAxes,
            ha="center", fontsize=10)
    ax.text(0.15, 0.85, "Low–High\n(Li −)", transform=ax.transAxes,
            ha="center", fontsize=10)
    ax.text(0.15, 0.10, "Low–Low\n(Li +)", transform=ax.transAxes,
            ha="center", fontsize=10)
    ax.text(0.65, 0.10, "High–Low\n(Li −)", transform=ax.transAxes,
            ha="center", fontsize=10)

    # ---- Axis labels ----
    ax.set_xlabel(f"{variable} (standardized)")
    ax.set_ylabel(f"Spatial lag of {variable}")

    # ---- Title with Global Moran ----
    ax.set_title(
        f"Moran Scatterplot – {variable} ({year})\n"
        f"Global Moran’s I = {I_global:.3f} (p = {p_global:.3f})",
        fontsize=11
    )

    ax.set_aspect("equal", adjustable="box")
#     ax.legend(loc="lower right", fontsize=9)

    plt.tight_layout()
    plt.savefig(out_path, dpi=300)
    plt.close()


In [154]:
INDICATORS = [
    "GDP_PC",
    "LOG_GDP_PC",
    "GDP_PC_PL_100",
    "GDP_PC_REG_100"
    
]

all_results = []

for year, df_year in gdf_wide.groupby("YEAR"):
    print(f"Processing year {year}...")
    
    year_dir = os.path.join(BASE_OUT, str(year))
    os.makedirs(year_dir, exist_ok=True)
    

    for var in INDICATORS:
        if var not in df_year.columns:
            continue

        lisa_gdf = run_lisa(df_year, var, w)
        lisa_gdf = gpd.GeoDataFrame(
            lisa_gdf,
            geometry="geometry",
            crs=gdf_wide.crs
        )

        # EXPORT TABLES 
        csv_path = os.path.join(year_dir, f"{var}_lisa.csv")
        gpkg_path = os.path.join(year_dir, f"{var}_lisa.gpkg")

        lisa_gdf.drop(columns="geometry").to_csv(csv_path, index=False)
        lisa_gdf.to_file(gpkg_path, driver="GPKG")

        # EXPORT MAP 
        map_path = os.path.join(year_dir, f"{var}_lisa_map.png")
        plot_lisa_map(lisa_gdf, var, year, map_path)
        
        ### EXPORT SCATTER PLOT
        scatter_path = os.path.join(year_dir, f"{var}_moran_scatter.png")
        plot_lisa_scatter(
            df_year,
            var,
            w,
            year,
            scatter_path
        )

        all_results.append(lisa_gdf)

print("LISA analysis completed.")


Processing year 2000...


C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose labe

Processing year 2001...


C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose labe

Processing year 2002...


...

Processing year 2022...


C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose labe

Processing year 2023...


C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right", fontsize=9)
C:\Users\jarem\AppData\Local\Temp\ipykernel_49672\1126631282.py:60: UserWarning: No artists with labels found to put in legend.  Note that artists whose labe

LISA analysis completed.
